d
# Example 4.1

This notebook shows Example 4.1 from the book showing how to use SQL on a US Flights Dataset dataset.

In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

Define a UDF to convert the date format into a legible format.

*Note*: the date is a string with year missing, so it might be difficult to do any queries using SQL `year()` function

In [0]:
def to_date_format_udf(d_str):
  l = [char for char in d_str]
  return "".join(l[0:2]) + "/" +  "".join(l[2:4]) + " " + " " +"".join(l[4:6]) + ":" + "".join(l[6:])

to_date_format_udf("02190925")

Register the UDF

In [0]:
spark.udf.register("to_date_format_udf", to_date_format_udf, StringType())

Read our US departure flight data

In [0]:
df = (spark.read.format("csv")
      .schema("date STRING, delay INT, distance INT, origin STRING, destination STRING")
      .option("header", "true")
      .option("path", "/databricks-datasets/learning-spark-v2/flights/departuredelays.csv")
      .load())

display(df)

Test our UDF

In [0]:
df.selectExpr("to_date_format_udf(date) as data_format").show(10, truncate=False)

Create a temporary view to which we can issue SQL queries

In [0]:
df.createOrReplaceTempView("us_delay_flights_tbl")

Cache Table so queries are expedient

In [0]:
%sql
CACHE TABLE us_delay_flights_tbl

Convert all `date` to `date_fm` so it's more eligible

Note: we are using UDF to convert it on the fly.

In [0]:
spark.sql("SELECT *, date, to_date_format_udf(date) AS date_fm FROM us_delay_flights_tbl").show(10, truncate=False)

In [0]:
spark.sql("SELECT COUNT(*) FROM us_delay_flights_tbl").show() # Keep case consistent for all SQL??

### Query 1:

 Find out all flights whose distance between origin and destination is greater than 1000

In [0]:
spark.sql("SELECT distance, origin, destination FROM us_delay_flights_tbl WHERE distance > 1000 ORDER BY distance DESC").show(10, truncate=False)

A DataFrame equivalent query

In [0]:
df.select("distance", "origin", "destination").where(col("distance") > 1000).orderBy(desc("distance")).show(10, truncate=False)

In [0]:
df.select("distance", "origin", "destination").where("distance > 1000").orderBy("distance", ascending=False).show(10)

In [0]:
df.select("distance", "origin", "destination").where("distance > 1000").orderBy(desc("distance")).show(10)

### Query 2:

 Find out all flights with 2 hour delays between San Francisco and Chicago

In [0]:
spark.sql("""
SELECT date, delay, origin, destination 
FROM us_delay_flights_tbl 
WHERE delay > 120 AND ORIGIN = 'SFO' AND DESTINATION = 'ORD' 
ORDER by delay DESC
""").show(10, truncate=False)

### Query 3:

A more complicated query in SQL, let's label all US flights originating from airports with _high_, _medium_, _low_, _no delays_, regardless of destinations.

In [0]:
spark.sql("""SELECT delay, origin, destination,
              CASE
                  WHEN delay > 360 THEN 'Very Long Delays'
                  WHEN delay > 120 AND delay < 360 THEN  'Long Delays '
                  WHEN delay > 60 AND delay < 120 THEN  'Short Delays'
                  WHEN delay > 0 and delay < 60  THEN   'Tolerable Delays'
                  WHEN delay = 0 THEN 'No Delays'
                  ELSE 'No Delays'
               END AS Flight_Delays
               FROM us_delay_flights_tbl
               ORDER BY origin, delay DESC""").show(10, truncate=False)


### Some Side Queries

In [0]:
df1 =  spark.sql("SELECT date, delay, origin, destination FROM us_delay_flights_tbl WHERE origin = 'SFO'")

In [0]:
df1.createOrReplaceGlobalTempView("us_origin_airport_SFO_tmp_view")

In [0]:
%sql
SELECT * FROM global_temp.us_origin_airport_SFO_tmp_view

In [0]:
%sql
DROP VIEW IF EXISTS global_temp.us_origin_airport_JFK_tmp_view

In [0]:
df2 = spark.sql("SELECT date, delay, origin, destination from us_delay_flights_tbl WHERE origin = 'JFK'")

In [0]:
df2.createOrReplaceTempView("us_origin_airport_JFK_tmp_view")

In [0]:
%sql
SELECT * FROM us_origin_airport_JFK_tmp_view

In [0]:
%sql
DROP VIEW IF EXISTS us_origin_airport_JFK_tmp_view

In [0]:
spark.catalog.listTables(dbName="global_temp")